In [1]:
!pip install openai python-dotenv gradio pandas langchain >installs.txt

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('company.db')
cursor = conn.cursor()

In [4]:
import sqlite3
import openai
from openai import OpenAI
import pandas as pd
import re
import gradio as gr

client = OpenAI()

def ask_sqlite_question(db_path, table, question, openai_api_key, model="gpt-3.5-turbo", temperature=0, max_tokens=256, frequency_penalty=0, presence_penalty=0):
    def get_table_schema():
        conn = sqlite3.connect(db_path)
        cursor = conn.execute(f"PRAGMA table_info({table})")
        col_attr = cursor.fetchall()
        col_names = [col[1] for col in col_attr]
        col_types = [col[2] for col in col_attr]
        col_joint = [f"{name} {ctype}" for name, ctype in zip(col_names, col_types)]
        schema = ", ".join(col_joint)
        conn.close()
        return schema, col_names

    def extract_sql_string(code):
        match = re.search(r"```sql\s*(.*?)\s*```", code, re.DOTALL)
        return match.group(1).strip() if match else ""

    schema, col_names = get_table_schema()
    
    system_template = """
    Given the following SQL table, your job is to write queries given a user’s request. \n
    CREATE TABLE {} ({}) \n
    """
    user_template = "Write a SQL query that returns - {}"

    system_message = system_template.format(table, schema)
    user_message = user_template.format(question)
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    openai.api_key = openai_api_key
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty
    )

    query = response.choices[0].message.content
    query = extract_sql_string(query)
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    conn.close()
    
    return query, result

def gradio_interface(db_path, table, openai_api_key, question):
    query, result = ask_sqlite_question(db_path, table, question, openai_api_key)
    # result_df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
    return query, result

db_path = "company.db"
table = "employees"
openai_api_key = "your_openai_api_key"

gr.Interface(
    fn=lambda query: gradio_interface(db_path, table, openai_api_key, query),
    inputs="text",
    outputs=[
        gr.Textbox(label="Generated SQL Query"),
        gr.Textbox(label="Query Result")
    ],
    title="SQLite Query Generator",
    description="Enter your query in plain English and get the corresponding SQL query and results from the database."
).launch()


/home/musab/text-to-sql/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [6]:
import sqlite3
import openai
from openai import OpenAI
import pandas as pd
import re
import gradio as gr

client = OpenAI()

def ask_sqlite_question(db_path, table, question, openai_api_key, model="gpt-3.5-turbo", temperature=0, max_tokens=256, frequency_penalty=0, presence_penalty=0):
    def get_table_schema():
        conn = sqlite3.connect(db_path)
        cursor = conn.execute(f"PRAGMA table_info({table})")
        col_attr = cursor.fetchall()
        col_names = [col[1] for col in col_attr]
        col_types = [col[2] for col in col_attr]
        col_joint = [f"{name} {ctype}" for name, ctype in zip(col_names, col_types)]
        schema = ", ".join(col_joint)
        conn.close()
        return schema, col_names

    def extract_sql_string(code):
        match = re.search(r"```sql\s*(.*?)\s*```", code, re.DOTALL)
        return match.group(1).strip() if match else ""

    schema, col_names = get_table_schema()
    
    system_template = """
    Given the following SQL table, your job is to write queries given a user’s request. \n
    CREATE TABLE {} ({}) \n
    """
    user_template = "Write a SQL query that returns - {}"

    system_message = system_template.format(table, schema)
    user_message = user_template.format(question)
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    openai.api_key = openai_api_key
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty
    )

    query = response.choices[0].message.content
    query = extract_sql_string(query)
    
    return query

def optimize_sql_query(query, openai_api_key, model="gpt-3.5-turbo", temperature=0, max_tokens=256, frequency_penalty=0, presence_penalty=0):
    system_message = "Given the following SQL query, analyze this query and suggest optimizations to improve performance:\n"
    user_message = query
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    openai.api_key = openai_api_key
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty
    )

    optimization_suggestions = response.choices[0].message.content.strip()
    
    return optimization_suggestions

def execute_sql_query(db_path, query):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    conn.close()
    return result

def gradio_interface(db_path, table, openai_api_key, question, query_suggestion):
    query = ask_sqlite_question(db_path, table, question, openai_api_key)
    result = execute_sql_query(db_path, query)
    optimization_suggestions = optimize_sql_query(query, openai_api_key)
    return query, result, optimization_suggestions

db_path = "company.db"
table = "employees"
openai_api_key = "your_openai_api_key"


gr.Interface(
    fn=lambda query, query_suggestion: gradio_interface(db_path, table, openai_api_key, query, query_suggestion),
    inputs=[
        gr.Textbox(label="Custom Query", placeholder="Enter your custom SQL query here..."),
    ],
    outputs=[
        gr.Textbox(label="Generated SQL Query"),
        gr.Textbox(label="Query Result"),
        gr.Textbox(label="Optimization Suggestions")
    ],
    title="SQLite Query Generator with Suggestions and Optimization",
    description="Enter your query in plain English or select a suggested query to get the corresponding SQL query, results from the database, and optimization suggestions."
).launch()


Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
